# Things to Do


- [Resume Here](#Resume-Here)

## Introduction

This notebook contains code and output of descriptive analyses for the 2000-2017 CC dataset after cleaning.

The objectives of this notebook are to:

- describe relationships between weather, particularly precipitation and temperature, and changes in population size 
- describe relationships between weather, particularly precipitation and temperature, and changes in population demographics

The metrics we examine are: .




##  Set up Python

First we will need to set up the python environment, importing the necessary packages and setting the display options.

[Top](#Table-of-Contents)

In [1]:
import pandas as pd
import numpy as np
import os, glob, logging
from summary_functions import *
from scipy import stats
from monthlit import *
from prettyprint import *


import plotly
import chart_studio.plotly as py
import plotly.figure_factory as ff
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

init_notebook_mode(connected=True)
# plotly.tools.set_config_file(world_readable=True)


# increase print limit
pd.options.display.max_rows = 99999
pd.options.display.max_columns = 50

### Setting File Locations

In [2]:
deviceDict = {'dataBig':{'source':'S:/Chris/TailDemography/TailDemography/weather data files'
                         ,'log':'S:/Chris/TailDemography/TailDemography/weather data files/logs'
                         ,'output':'S:/Chris/TailDemography/TailDemography/weather data files/outputFiles/'},
              'silverSurfer':{'source':'C:\\Users\\craga_eowcrpe\\Google Drive\\TailDemography\\weather data files/outputFiles'
                              ,'log':'C:\\Users\\craga_eowcrpe\\Google Drive\\TailDemography\\weather data files/logs'
                              ,'output':'C:\\Users\\craga_eowcrpe\\Google Drive\\TailDemography\\weather data files/outputFiles'}
              ,'dataPers':{'source':'C:/Users/Christopher/Google Drive/TailDemography/weather data files'
                           ,'log': 'C:\\Users\\craga_eowcrpe\\Google Drive\\TailDemography\\weather data files/logs'
                           ,'output':'C:/Users/Christopher/Google Drive/TailDemography/weather data files/outputFiles'}
             ,'gandolf':{'source':'C:/Users/craga/Google Drive/TailDemography/weather data files'
                           ,'log': 'C:/Users/craga/Google Drive/TailDemography/weather data files/logs'
                           ,'output':'C:/Users/craga/Google Drive/TailDemography/weather data files/outputFiles'}}

### Choose Device

In [3]:
device = deviceDict['gandolf']
device

{'source': 'C:/Users/craga/Google Drive/TailDemography/weather data files',
 'log': 'C:/Users/craga/Google Drive/TailDemography/weather data files/logs',
 'output': 'C:/Users/craga/Google Drive/TailDemography/weather data files/outputFiles'}

# Source Data


### Logging

In [4]:
logging.basicConfig(filename=device['log']+'Desriptive Analyses.log'
                    , filemode='a',
                    format='%(funcName)s - %(levelname)s - %(message)s - %(asctime)s', level=logging.DEBUG)

## Functions

This section contains functions that were created for this notebook.

- [distribution](#distribution) #delete this we will use scipy stats describe instead
- [monthlit](#monthlit)
- [description](#description)
- [vocab_run](#vocab_run)

### distribution
[Back to Top](#TOC)

[Back to Functions](#Functions)

*distribution* takes a series or list of numeric objects, *x*, and returns descriptive stats of x including
        n, minimum, maximum, median, sIQR, mean, and stdev
    
Here are a few examples of how *distribution* works.

In [5]:
foo = [0,1,2,'r']
distribution(foo)

In [6]:
bar = [0,1,2]
distribution(bar)

,n,minimum,maximum,median,siqr,mean,stdev
0,3,0,2,1.0,0.5,1.0,1.0


[Back to Functions](#Functions)

## monthlit
[Back to Top](#TOC)

[Back to Functions](#Functions)

Here are a few examples of how _monthlit_ works.

In [7]:
dates = pd.DataFrame(data={'dates':['2018-12-9','2019-8-5', '2017/7/4',np.nan,None]})
dates.dates = pd.to_datetime(dates.dates)
dates

,dates
0,2018-12-09
1,2019-08-05
2,2017-07-04
3,NaT
4,NaT


In [8]:
monthlit(dates.dates.dt.month[0])

'Dec'

In [9]:
dates.dates.dt.month.apply(monthlit)

0    Dec
1    Aug
2    Jul
3    NaN
4    NaN
Name: dates, dtype: object

[Back to Functions](#Functions)

## description
[Back to Top](#TOC)

[Back to Functions](#Functions)

In [10]:
def description(x,variable,percentage=False):
    if percentage:
            res = x[variable].describe()
            res[['mean','std','min','25%','50%','75%','max']] = res[['mean','std','min','25%','50%','75%','max']]\
            .apply(lambda x:x*100) 
#Need to Add CI calculation to this function
#             meanCI = 'not calculated'
    else:
        res = x[variable].describe() 
    res['siqr'] = (res['75%']-res['25%'])/2
    res['meanCI'] = 'not calculated'
    return res

### vocab_run
[Back to Top](#TOC)

[Back to Functions](#Functions)

*vocab_run* takes a list, joins its the first the elements with a separator placing a different separator between
     the penultimate and final members of the list and returns the result as a string
     :param x: a list of strings to be concatenated
     :param connector_dict: a dictionary with keys describing the size of the list and values indicating the type of
     connectors separate the list elements.
    
Here are a few examples of how *vocab_run* works.

In [11]:
print("Could you bring some {} please?".format(vocab_run(['foo','bar','stuffkins'])))

Could you bring some foo, bar and stuffkins please?


In [12]:
print("You can either have {}.  You'll have to make a choice."\
      .format(vocab_run(['foo','bar','stuffkins'],connector_dict={1: None, 2: ' or ', 'run': ', '})))

You can either have foo, bar or stuffkins.  You'll have to make a choice.


[Back to Functions](#Functions)

We'll display all files in the source folder with the prefix _'cleaned CC data 2000-2017'_. The file names will be saved in a variable, _mysourcefiles_.

## Get Data
[Top](#TOC)

Here we can set the locations from which we get data and to which we export it.

In [13]:
os.chdir(device['source'])
mysourcefiles = glob.glob('*_weather*.csv')
mysourcefiles

['paradise_weatherdata.csv', 'portal_weatherdata.csv']

In [14]:
def getweatherdata(afile,sourcename):
    tmp = pd.read_csv(afile)
    tmp['source'] = sourcename
    return tmp

Get weather data

In [15]:
df = pd.concat([getweatherdata(afile,afile.split('_')[0]) for afile in mysourcefiles]).drop(columns = 'Unnamed: 0')

Get population data.

In [16]:
df_pop = pd.read_csv('C:/Users/craga/Google Drive/TailDemography/outputFiles/Descriptive/population size 56 mm plus.csv')
# df_pop = pd.read_csv('C:/Users/craga/Google Drive/TailDemography/outputFiles/Descriptive/population size 56 plus.csv')
df_pop.head()

,year,sex,liznumber,liznumberYear,propMale,propFemale
0,2000,f,65,124,NaN,0.524194
1,2000,m,59,124,0.475806,NaN
2,2001,f,47,92,NaN,0.510870
3,2001,m,45,92,0.489130,NaN
4,2002,f,51,97,NaN,0.525773


# Analyze Data
[Top](#TOC)

We will first examine the range and distribution of number of variables in our data set:


In [17]:
# Weather for a year includes weather since the last collection date of the previous calendar year 
seasons={'Dec':'winter','Jan':'winter','Feb':'winter',
         'Mar':'spring','Apr':'spring','May':'spring',
         'Jun':'summer','Jul':'summer','Aug':'summer',
         'Sept':'fall','Oct':'fall','Nov':'fall'}

Split analysis up:
- Analysis 1
    - weather in the previous 365 days relative to the first date of collection/sighting for the current calendar year
    - additional factor would be population for previous calendar year (year -1)
- Analysis 2 (Skip this for now)
    - weather in the previous 365 days relative to the first date of collection/sighting for the current calendar year
    - additonal factor would be populationi in the the current calendar year (year 0)
    - dv: population in (year 1 through year x)
- Analysis 3,4,5
    - IV
        - population in year -1
        - onset of monsoon in year 0
        - precipitation in summer
        - interaction ?
    - DV
        - population in year 1
        - age/size structure in year 1 (looking for 45mm to 65mm)
        - sex ratio in year 1

In [18]:
# This could be used to generate season-level weather data (use season dates) - Chris
# This could also be used to approximate the start of the monsoon season
# Check historical data in May and June to identify in notes when the first juvenile were spotted - (George and Chris)
## Look for correlates in the data
# Use SWRS data to identify start of monsoons (George to get SWRS data)
# what other precipitation and temperature in the NOAA data set have been used for this (George and Chris to check the lit)
df['month'] = df.month.apply(monthlit)
df['season'] = df.month.apply(lambda x: seasons[x])
df_season = pd.DataFrame(df.groupby(['source','year','season'])['PRCP','SNOW','TMAX','TMIN','TAVG'].describe())[1:-1]
df_season.columns = [' '.join(col).strip() for col in df_season.columns.values]
df_season = df_season.reset_index()

In [19]:
df_season['year-season'] = df_season.year.astype(str) + '-' + df_season.season
df_season

,source,year,season,PRCP count,PRCP mean,PRCP std,PRCP min,PRCP 25%,PRCP 50%,PRCP 75%,PRCP max,SNOW count,SNOW mean,SNOW std,SNOW min,SNOW 25%,SNOW 50%,SNOW 75%,SNOW max,TMAX count,TMAX mean,TMAX std,TMAX min,TMAX 25%,TMAX 50%,TMAX 75%,TMAX max,TMIN count,TMIN mean,TMIN std,TMIN min,TMIN 25%,TMIN 50%,TMIN 75%,TMIN max,TAVG count,TAVG mean,TAVG std,TAVG min,TAVG 25%,TAVG 50%,TAVG 75%,TAVG max,year-season
0,paradise,2007,summer,2.0,3.865000,1.350574,2.91,3.3875,3.865,4.3425,4.82,2.0,0.000000,0.000000,0.0,0.00,0.0,0.00,0.0,2.0,86.500000,3.676955,83.9,85.200,86.50,87.800,89.1,2.0,59.050000,0.212132,58.9,58.975,59.05,59.125,59.2,2.0,72.800000,1.979899,71.4,72.100,72.80,73.500,74.2,2007-summer
1,paradise,2007,winter,1.0,2.510000,NaN,2.51,2.5100,2.510,2.5100,2.51,1.0,0.000000,NaN,0.0,0.00,0.0,0.00,0.0,1.0,51.300000,NaN,51.3,51.300,51.30,51.300,51.3,1.0,23.800000,NaN,23.8,23.800,23.80,23.800,23.8,1.0,37.600000,NaN,37.6,37.600,37.60,37.600,37.6,2007-winter
2,paradise,2008,fall,3.0,0.880000,0.810740,0.27,0.4200,0.570,1.1850,1.80,3.0,0.000000,0.000000,0.0,0.00,0.0,0.00,0.0,3.0,72.100000,7.351190,63.9,69.100,74.30,76.200,78.1,3.0,40.000000,9.153688,31.0,35.350,39.70,44.500,49.3,3.0,56.033333,8.192883,47.4,52.200,57.00,60.350,63.7,2008-fall
3,paradise,2008,summer,2.0,8.440000,7.071068,3.44,5.9400,8.440,10.9400,13.44,2.0,0.000000,0.000000,0.0,0.00,0.0,0.00,0.0,2.0,82.500000,0.848528,81.9,82.200,82.50,82.800,83.1,2.0,58.450000,0.919239,57.8,58.125,58.45,58.775,59.1,2.0,70.500000,0.848528,69.9,70.200,70.50,70.800,71.1,2008-summer
4,paradise,2008,winter,1.0,1.200000,NaN,1.20,1.2000,1.200,1.2000,1.20,1.0,1.300000,NaN,1.3,1.30,1.3,1.30,1.3,1.0,55.300000,NaN,55.3,55.300,55.30,55.300,55.3,1.0,26.900000,NaN,26.9,26.900,26.90,26.900,26.9,1.0,41.100000,NaN,41.1,41.100,41.10,41.100,41.1,2008-winter
5,paradise,2009,fall,3.0,1.650000,0.546717,1.02,1.4750,1.930,1.9650,2.00,3.0,1.933333,3.348632,0.0,0.00,0.0,2.90,5.8,3.0,72.600000,8.444525,64.7,68.150,71.60,76.550,81.5,3.0,41.266667,9.600174,31.7,36.450,41.20,46.050,50.9,3.0,56.933333,9.011844,48.2,52.300,56.40,61.300,66.2,2009-fall
6,paradise,2009,spring,3.0,0.466667,0.411987,0.00,0.3100,0.620,0.7000,0.78,3.0,0.000000,0.000000,0.0,0.00,0.0,0.00,0.0,3.0,73.266667,8.447682,66.4,68.550,70.70,76.700,82.7,3.0,39.666667,8.220908,33.5,35.000,36.50,42.750,49.0,3.0,56.500000,8.337266,50.0,51.800,53.60,59.750,65.9,2009-spring
7,paradise,2009,summer,3.0,1.970000,1.165032,0.66,1.5100,2.360,2.6250,2.89,3.0,0.000000,0.000000,0.0,0.00,0.0,0.00,0.0,3.0,87.600000,2.433105,84.8,86.800,88.80,89.000,89.2,3.0,57.433333,4.271222,52.6,55.800,59.00,59.850,60.7,3.0,72.533333,3.365016,68.7,71.300,73.90,74.450,75.0,2009-summer
8,paradise,2009,winter,3.0,1.106667,0.603435,0.70,0.7600,0.820,1.3100,1.80,3.0,5.400000,3.966106,1.0,3.75,6.5,7.60,8.7,3.0,55.800000,6.509224,49.1,52.650,56.20,59.150,62.1,3.0,25.433333,0.960902,24.4,25.000,25.60,25.950,26.3,3.0,40.600000,3.758989,36.7,38.800,40.90,42.550,44.2,2009-winter
9,paradise,2010,fall,3.0,0.743333,0.610109,0.14,0.4350,0.730,1.0450,1.36,3.0,0.000000,0.000000,0.0,0.00,0.0,0.00,0.0,3.0,72.833333,10.327310,61.7,68.200,74.70,78.400,82.1,3.0,40.900000,12.517588,27.7,35.050,42.40,47.500,52.6,3.0,56.866667,11.437803,44.7,51.600,58.50,62.950,67.4,2010-fall


In [20]:
df_annual = pd.DataFrame(df.groupby(['source','year'])['PRCP','SNOW','TMAX','TMIN','TAVG'].describe())[1:-1]
df_annual.columns = [' '.join(col).strip() for col in df_annual.columns.values]
df_annual = df_annual.reset_index().sort_values('year')
df_annual

,source,year,PRCP count,PRCP mean,PRCP std,PRCP min,PRCP 25%,PRCP 50%,PRCP 75%,PRCP max,SNOW count,SNOW mean,SNOW std,SNOW min,SNOW 25%,SNOW 50%,SNOW 75%,SNOW max,TMAX count,TMAX mean,TMAX std,TMAX min,TMAX 25%,TMAX 50%,TMAX 75%,TMAX max,TMIN count,TMIN mean,TMIN std,TMIN min,TMIN 25%,TMIN 50%,TMIN 75%,TMIN max,TAVG count,TAVG mean,TAVG std,TAVG min,TAVG 25%,TAVG 50%,TAVG 75%,TAVG max
10,portal,2000,11.0,1.713636,2.477564,0.00,0.0250,0.520,2.0600,7.81,11.0,0.000000,0.000000,0.0,0.0,0.0,0.000,0.0,11.0,71.500000,12.671622,54.6,61.700,67.70,84.950,86.5,11.0,37.536364,12.668881,23.1,26.750,35.40,48.650,55.6,11.0,54.518182,12.407967,39.5,43.900,54.30,67.100,71.0
11,portal,2001,12.0,1.295000,1.162470,0.37,0.5275,0.640,1.8650,3.81,12.0,0.950000,2.251666,0.0,0.0,0.0,0.000,6.6,11.0,69.954545,13.434386,50.7,59.700,72.00,82.000,88.1,10.0,35.330000,11.050696,20.9,27.150,34.60,42.000,54.2,10.0,51.750000,11.763338,36.6,42.775,50.95,61.275,69.1
12,portal,2002,12.0,1.029167,1.404593,0.00,0.0525,0.645,1.3100,5.07,12.0,0.241667,0.583809,0.0,0.0,0.0,0.000,1.8,12.0,72.183333,13.629302,50.7,61.825,73.15,82.925,91.5,12.0,38.091667,12.724883,23.3,27.525,37.75,50.350,57.8,12.0,55.133333,12.957576,37.2,44.650,55.50,66.500,72.4
13,portal,2003,12.0,0.963333,0.955485,0.05,0.3400,0.500,1.7250,2.57,12.0,0.091667,0.317543,0.0,0.0,0.0,0.000,1.1,12.0,73.141667,13.112482,55.0,62.050,74.10,83.850,92.2,12.0,38.225000,12.503754,21.2,28.025,37.50,48.925,56.4,12.0,55.691667,12.695129,38.1,44.700,55.80,66.800,74.2
14,portal,2004,12.0,1.757500,1.034427,0.50,0.9175,1.620,2.2750,3.83,11.0,0.409091,0.970005,0.0,0.0,0.0,0.000,3.0,10.0,66.550000,11.667833,52.6,55.475,67.85,76.550,82.4,10.0,35.720000,11.631261,22.6,26.400,34.35,43.675,53.7,9.0,49.333333,10.694625,37.7,39.400,50.60,54.600,67.4
15,portal,2005,12.0,1.884167,1.867115,0.00,0.2075,1.135,3.3525,4.89,10.0,0.270000,0.853815,0.0,0.0,0.0,0.000,2.7,11.0,73.727273,13.214544,53.5,63.950,73.70,83.000,91.2,11.0,39.409091,11.049838,26.6,30.400,37.60,48.550,55.3,11.0,56.572727,11.773028,40.8,46.100,55.00,67.200,73.3
16,portal,2006,11.0,2.144545,2.857133,0.12,0.2850,0.770,3.2900,8.59,11.0,0.409091,1.356801,0.0,0.0,0.0,0.000,4.5,10.0,73.210000,12.916265,53.5,63.800,73.50,83.650,91.1,9.0,40.766667,12.964760,22.0,30.900,39.50,53.000,58.1,9.0,58.077778,11.956460,40.8,47.300,57.00,67.700,73.2
17,portal,2007,12.0,1.807500,1.792236,0.03,0.4850,1.135,2.8150,5.76,12.0,1.175000,3.185799,0.0,0.0,0.0,0.250,11.1,11.0,71.681818,14.050823,47.5,62.750,76.00,82.250,88.4,11.0,39.245455,13.608847,21.8,26.350,39.70,50.500,57.5,11.0,55.472727,13.650208,34.7,44.550,57.80,67.600,72.4
18,portal,2008,11.0,1.741818,3.014614,0.00,0.0100,0.610,1.6750,9.91,10.0,0.100000,0.316228,0.0,0.0,0.0,0.000,1.0,10.0,74.040000,10.226023,56.0,68.050,75.60,80.475,89.4,10.0,40.660000,11.961159,26.3,29.050,39.70,50.050,57.1,10.0,57.350000,10.742258,41.8,48.550,58.00,67.250,70.2
0,paradise,2008,6.0,3.453333,5.019799,0.27,0.7275,1.500,3.0300,13.44,6.0,0.216667,0.530723,0.0,0.0,0.0,0.000,1.3,6.0,72.766667,10.997212,55.3,66.500,76.20,80.950,83.1,6.0,43.966667,13.612005,26.9,33.175,44.50,55.675,59.1,6.0,58.366667,12.197322,41.1,49.800,60.35,68.350,71.1


## Population Size

Can we predict the change in population size using the prvious year's weather?
First let's make a new data set that will allow us to vizualize the potential relationship between precipitation and population size.

In [21]:
df_reg_annual = df_annual.merge(df_pop.loc[df_pop.sex=='f'].drop(columns=['propMale','sex','liznumber']),on = ['year'],how='left')
df_reg_annual.head()

,source,year,PRCP count,PRCP mean,PRCP std,PRCP min,PRCP 25%,PRCP 50%,PRCP 75%,PRCP max,SNOW count,SNOW mean,SNOW std,SNOW min,SNOW 25%,SNOW 50%,SNOW 75%,SNOW max,TMAX count,TMAX mean,TMAX std,TMAX min,TMAX 25%,TMAX 50%,TMAX 75%,TMAX max,TMIN count,TMIN mean,TMIN std,TMIN min,TMIN 25%,TMIN 50%,TMIN 75%,TMIN max,TAVG count,TAVG mean,TAVG std,TAVG min,TAVG 25%,TAVG 50%,TAVG 75%,TAVG max,liznumberYear,propFemale
0,portal,2000,11.0,1.713636,2.477564,0.00,0.0250,0.520,2.060,7.81,11.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,11.0,71.500000,12.671622,54.6,61.700,67.70,84.950,86.5,11.0,37.536364,12.668881,23.1,26.750,35.40,48.650,55.6,11.0,54.518182,12.407967,39.5,43.900,54.30,67.100,71.0,124,0.524194
1,portal,2001,12.0,1.295000,1.162470,0.37,0.5275,0.640,1.865,3.81,12.0,0.950000,2.251666,0.0,0.0,0.0,0.0,6.6,11.0,69.954545,13.434386,50.7,59.700,72.00,82.000,88.1,10.0,35.330000,11.050696,20.9,27.150,34.60,42.000,54.2,10.0,51.750000,11.763338,36.6,42.775,50.95,61.275,69.1,92,0.510870
2,portal,2002,12.0,1.029167,1.404593,0.00,0.0525,0.645,1.310,5.07,12.0,0.241667,0.583809,0.0,0.0,0.0,0.0,1.8,12.0,72.183333,13.629302,50.7,61.825,73.15,82.925,91.5,12.0,38.091667,12.724883,23.3,27.525,37.75,50.350,57.8,12.0,55.133333,12.957576,37.2,44.650,55.50,66.500,72.4,97,0.525773
3,portal,2003,12.0,0.963333,0.955485,0.05,0.3400,0.500,1.725,2.57,12.0,0.091667,0.317543,0.0,0.0,0.0,0.0,1.1,12.0,73.141667,13.112482,55.0,62.050,74.10,83.850,92.2,12.0,38.225000,12.503754,21.2,28.025,37.50,48.925,56.4,12.0,55.691667,12.695129,38.1,44.700,55.80,66.800,74.2,63,0.555556
4,portal,2004,12.0,1.757500,1.034427,0.50,0.9175,1.620,2.275,3.83,11.0,0.409091,0.970005,0.0,0.0,0.0,0.0,3.0,10.0,66.550000,11.667833,52.6,55.475,67.85,76.550,82.4,10.0,35.720000,11.631261,22.6,26.400,34.35,43.675,53.7,9.0,49.333333,10.694625,37.7,39.400,50.60,54.600,67.4,54,0.555556


In [22]:
df_reg_season = df_season.merge(df_pop.loc[df_pop.sex=='f'].drop(columns=['propMale','sex','liznumber']),on = ['year'],how='left')
df_reg_season.head()

,source,year,season,PRCP count,PRCP mean,PRCP std,PRCP min,PRCP 25%,PRCP 50%,PRCP 75%,PRCP max,SNOW count,SNOW mean,SNOW std,SNOW min,SNOW 25%,SNOW 50%,SNOW 75%,SNOW max,TMAX count,TMAX mean,TMAX std,TMAX min,TMAX 25%,TMAX 50%,TMAX 75%,TMAX max,TMIN count,TMIN mean,TMIN std,TMIN min,TMIN 25%,TMIN 50%,TMIN 75%,TMIN max,TAVG count,TAVG mean,TAVG std,TAVG min,TAVG 25%,TAVG 50%,TAVG 75%,TAVG max,year-season,liznumberYear,propFemale
0,paradise,2007,summer,2.0,3.865,1.350574,2.91,3.3875,3.865,4.3425,4.82,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,86.5,3.676955,83.9,85.2,86.5,87.8,89.1,2.0,59.05,0.212132,58.9,58.975,59.05,59.125,59.2,2.0,72.800000,1.979899,71.4,72.1,72.8,73.50,74.2,2007-summer,51,0.588235
1,paradise,2007,winter,1.0,2.510,NaN,2.51,2.5100,2.510,2.5100,2.51,1.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,1.0,51.3,NaN,51.3,51.3,51.3,51.3,51.3,1.0,23.80,NaN,23.8,23.800,23.80,23.800,23.8,1.0,37.600000,NaN,37.6,37.6,37.6,37.60,37.6,2007-winter,51,0.588235
2,paradise,2008,fall,3.0,0.880,0.810740,0.27,0.4200,0.570,1.1850,1.80,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,72.1,7.351190,63.9,69.1,74.3,76.2,78.1,3.0,40.00,9.153688,31.0,35.350,39.70,44.500,49.3,3.0,56.033333,8.192883,47.4,52.2,57.0,60.35,63.7,2008-fall,67,0.522388
3,paradise,2008,summer,2.0,8.440,7.071068,3.44,5.9400,8.440,10.9400,13.44,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,82.5,0.848528,81.9,82.2,82.5,82.8,83.1,2.0,58.45,0.919239,57.8,58.125,58.45,58.775,59.1,2.0,70.500000,0.848528,69.9,70.2,70.5,70.80,71.1,2008-summer,67,0.522388
4,paradise,2008,winter,1.0,1.200,NaN,1.20,1.2000,1.200,1.2000,1.20,1.0,1.3,NaN,1.3,1.3,1.3,1.3,1.3,1.0,55.3,NaN,55.3,55.3,55.3,55.3,55.3,1.0,26.90,NaN,26.9,26.900,26.90,26.900,26.9,1.0,41.100000,NaN,41.1,41.1,41.1,41.10,41.1,2008-winter,67,0.522388


In [23]:
#Drop paradise
df_reg_annual['popinYearplus1'] = df_reg_annual.groupby('source').liznumberYear.shift(1)
df_reg_annual['popinYearless1'] = df_reg_annual.groupby('source').liznumberYear.shift(-1)
df_reg_annual['popinYearless2'] = df_reg_annual.groupby('source').liznumberYear.shift(-2)
df_reg_annual['popinYearless3'] = df_reg_annual.groupby('source').liznumberYear.shift(-3)
df_reg_annual['popinYearless4'] = df_reg_annual.groupby('source').liznumberYear.shift(-4)
df_reg_annual['popinYearless5'] = df_reg_annual.groupby('source').liznumberYear.shift(-5)
df_reg_annual

,source,year,PRCP count,PRCP mean,PRCP std,PRCP min,PRCP 25%,PRCP 50%,PRCP 75%,PRCP max,SNOW count,SNOW mean,SNOW std,SNOW min,SNOW 25%,SNOW 50%,SNOW 75%,SNOW max,TMAX count,TMAX mean,TMAX std,TMAX min,TMAX 25%,TMAX 50%,TMAX 75%,TMAX max,TMIN count,TMIN mean,TMIN std,TMIN min,TMIN 25%,TMIN 50%,TMIN 75%,TMIN max,TAVG count,TAVG mean,TAVG std,TAVG min,TAVG 25%,TAVG 50%,TAVG 75%,TAVG max,liznumberYear,propFemale,popinYearplus1,popinYearless1,popinYearless2,popinYearless3,popinYearless4,popinYearless5
0,portal,2000,11.0,1.713636,2.477564,0.00,0.0250,0.520,2.0600,7.81,11.0,0.000000,0.000000,0.0,0.0,0.0,0.000,0.0,11.0,71.500000,12.671622,54.6,61.700,67.70,84.950,86.5,11.0,37.536364,12.668881,23.1,26.750,35.40,48.650,55.6,11.0,54.518182,12.407967,39.5,43.900,54.30,67.100,71.0,124,0.524194,NaN,92.0,97.0,63.0,54.0,60.0
1,portal,2001,12.0,1.295000,1.162470,0.37,0.5275,0.640,1.8650,3.81,12.0,0.950000,2.251666,0.0,0.0,0.0,0.000,6.6,11.0,69.954545,13.434386,50.7,59.700,72.00,82.000,88.1,10.0,35.330000,11.050696,20.9,27.150,34.60,42.000,54.2,10.0,51.750000,11.763338,36.6,42.775,50.95,61.275,69.1,92,0.510870,124.0,97.0,63.0,54.0,60.0,2.0
2,portal,2002,12.0,1.029167,1.404593,0.00,0.0525,0.645,1.3100,5.07,12.0,0.241667,0.583809,0.0,0.0,0.0,0.000,1.8,12.0,72.183333,13.629302,50.7,61.825,73.15,82.925,91.5,12.0,38.091667,12.724883,23.3,27.525,37.75,50.350,57.8,12.0,55.133333,12.957576,37.2,44.650,55.50,66.500,72.4,97,0.525773,92.0,63.0,54.0,60.0,2.0,51.0
3,portal,2003,12.0,0.963333,0.955485,0.05,0.3400,0.500,1.7250,2.57,12.0,0.091667,0.317543,0.0,0.0,0.0,0.000,1.1,12.0,73.141667,13.112482,55.0,62.050,74.10,83.850,92.2,12.0,38.225000,12.503754,21.2,28.025,37.50,48.925,56.4,12.0,55.691667,12.695129,38.1,44.700,55.80,66.800,74.2,63,0.555556,97.0,54.0,60.0,2.0,51.0,67.0
4,portal,2004,12.0,1.757500,1.034427,0.50,0.9175,1.620,2.2750,3.83,11.0,0.409091,0.970005,0.0,0.0,0.0,0.000,3.0,10.0,66.550000,11.667833,52.6,55.475,67.85,76.550,82.4,10.0,35.720000,11.631261,22.6,26.400,34.35,43.675,53.7,9.0,49.333333,10.694625,37.7,39.400,50.60,54.600,67.4,54,0.555556,63.0,60.0,2.0,51.0,67.0,54.0
5,portal,2005,12.0,1.884167,1.867115,0.00,0.2075,1.135,3.3525,4.89,10.0,0.270000,0.853815,0.0,0.0,0.0,0.000,2.7,11.0,73.727273,13.214544,53.5,63.950,73.70,83.000,91.2,11.0,39.409091,11.049838,26.6,30.400,37.60,48.550,55.3,11.0,56.572727,11.773028,40.8,46.100,55.00,67.200,73.3,60,0.483333,54.0,2.0,51.0,67.0,54.0,45.0
6,portal,2006,11.0,2.144545,2.857133,0.12,0.2850,0.770,3.2900,8.59,11.0,0.409091,1.356801,0.0,0.0,0.0,0.000,4.5,10.0,73.210000,12.916265,53.5,63.800,73.50,83.650,91.1,9.0,40.766667,12.964760,22.0,30.900,39.50,53.000,58.1,9.0,58.077778,11.956460,40.8,47.300,57.00,67.700,73.2,2,0.500000,60.0,51.0,67.0,54.0,45.0,5.0
7,portal,2007,12.0,1.807500,1.792236,0.03,0.4850,1.135,2.8150,5.76,12.0,1.175000,3.185799,0.0,0.0,0.0,0.250,11.1,11.0,71.681818,14.050823,47.5,62.750,76.00,82.250,88.4,11.0,39.245455,13.608847,21.8,26.350,39.70,50.500,57.5,11.0,55.472727,13.650208,34.7,44.550,57.80,67.600,72.4,51,0.588235,2.0,67.0,54.0,45.0,5.0,51.0
8,portal,2008,11.0,1.741818,3.014614,0.00,0.0100,0.610,1.6750,9.91,10.0,0.100000,0.316228,0.0,0.0,0.0,0.000,1.0,10.0,74.040000,10.226023,56.0,68.050,75.60,80.475,89.4,10.0,40.660000,11.961159,26.3,29.050,39.70,50.050,57.1,10.0,57.350000,10.742258,41.8,48.550,58.00,67.250,70.2,67,0.522388,51.0,54.0,45.0,5.0,51.0,54.0
9,paradise,2008,6.0,3.453333,5.019799,0.27,0.7275,1.500,3.0300,13.44,6.0,0.216667,0.530723,0.0,0.0,0.0,0.000,1.3,6.0,72.766667,10.997212,55.3,66.500,76.20,80.950,83.1,6.0,43.966667,13.612005,26.9,33.175,44.50,55.675,59.1,6.0,58.366667,12.197322,41.1,49.800,60.35,68.350,71.1,67,0.522388,NaN,54.0,45.0,5.0,51.0,54.0


In [24]:
#Drop paradise
df_reg_season['popinYearplus1'] = df_reg_season.groupby('source').liznumberYear.shift(1)
df_reg_season['popinYearless1'] = df_reg_season.groupby('source').liznumberYear.shift(-1)
df_reg_season['popinYearless2'] = df_reg_season.groupby('source').liznumberYear.shift(-2)
df_reg_season['popinYearless3'] = df_reg_season.groupby('source').liznumberYear.shift(-3)
df_reg_season['popinYearless4'] = df_reg_season.groupby('source').liznumberYear.shift(-4)
df_reg_season['popinYearless5'] = df_reg_season.groupby('source').liznumberYear.shift(-5)
df_reg_season

,source,year,season,PRCP count,PRCP mean,PRCP std,PRCP min,PRCP 25%,PRCP 50%,PRCP 75%,PRCP max,SNOW count,SNOW mean,SNOW std,SNOW min,SNOW 25%,SNOW 50%,SNOW 75%,SNOW max,TMAX count,TMAX mean,TMAX std,TMAX min,TMAX 25%,TMAX 50%,...,TMIN count,TMIN mean,TMIN std,TMIN min,TMIN 25%,TMIN 50%,TMIN 75%,TMIN max,TAVG count,TAVG mean,TAVG std,TAVG min,TAVG 25%,TAVG 50%,TAVG 75%,TAVG max,year-season,liznumberYear,propFemale,popinYearplus1,popinYearless1,popinYearless2,popinYearless3,popinYearless4,popinYearless5
0,paradise,2007,summer,2.0,3.865000,1.350574,2.91,3.3875,3.865,4.3425,4.82,2.0,0.000000,0.000000,0.0,0.00,0.0,0.00,0.0,2.0,86.500000,3.676955,83.9,85.200,86.50,...,2.0,59.050000,0.212132,58.9,58.975,59.05,59.125,59.2,2.0,72.800000,1.979899,71.4,72.100,72.80,73.500,74.2,2007-summer,51,0.588235,NaN,51.0,67.0,67.0,67.0,54.0
1,paradise,2007,winter,1.0,2.510000,NaN,2.51,2.5100,2.510,2.5100,2.51,1.0,0.000000,NaN,0.0,0.00,0.0,0.00,0.0,1.0,51.300000,NaN,51.3,51.300,51.30,...,1.0,23.800000,NaN,23.8,23.800,23.80,23.800,23.8,1.0,37.600000,NaN,37.6,37.600,37.60,37.600,37.6,2007-winter,51,0.588235,51.0,67.0,67.0,67.0,54.0,54.0
2,paradise,2008,fall,3.0,0.880000,0.810740,0.27,0.4200,0.570,1.1850,1.80,3.0,0.000000,0.000000,0.0,0.00,0.0,0.00,0.0,3.0,72.100000,7.351190,63.9,69.100,74.30,...,3.0,40.000000,9.153688,31.0,35.350,39.70,44.500,49.3,3.0,56.033333,8.192883,47.4,52.200,57.00,60.350,63.7,2008-fall,67,0.522388,51.0,67.0,67.0,54.0,54.0,54.0
3,paradise,2008,summer,2.0,8.440000,7.071068,3.44,5.9400,8.440,10.9400,13.44,2.0,0.000000,0.000000,0.0,0.00,0.0,0.00,0.0,2.0,82.500000,0.848528,81.9,82.200,82.50,...,2.0,58.450000,0.919239,57.8,58.125,58.45,58.775,59.1,2.0,70.500000,0.848528,69.9,70.200,70.50,70.800,71.1,2008-summer,67,0.522388,67.0,67.0,54.0,54.0,54.0,54.0
4,paradise,2008,winter,1.0,1.200000,NaN,1.20,1.2000,1.200,1.2000,1.20,1.0,1.300000,NaN,1.3,1.30,1.3,1.30,1.3,1.0,55.300000,NaN,55.3,55.300,55.30,...,1.0,26.900000,NaN,26.9,26.900,26.90,26.900,26.9,1.0,41.100000,NaN,41.1,41.100,41.10,41.100,41.1,2008-winter,67,0.522388,67.0,54.0,54.0,54.0,54.0,45.0
5,paradise,2009,fall,3.0,1.650000,0.546717,1.02,1.4750,1.930,1.9650,2.00,3.0,1.933333,3.348632,0.0,0.00,0.0,2.90,5.8,3.0,72.600000,8.444525,64.7,68.150,71.60,...,3.0,41.266667,9.600174,31.7,36.450,41.20,46.050,50.9,3.0,56.933333,9.011844,48.2,52.300,56.40,61.300,66.2,2009-fall,54,0.537037,67.0,54.0,54.0,54.0,45.0,45.0
6,paradise,2009,spring,3.0,0.466667,0.411987,0.00,0.3100,0.620,0.7000,0.78,3.0,0.000000,0.000000,0.0,0.00,0.0,0.00,0.0,3.0,73.266667,8.447682,66.4,68.550,70.70,...,3.0,39.666667,8.220908,33.5,35.000,36.50,42.750,49.0,3.0,56.500000,8.337266,50.0,51.800,53.60,59.750,65.9,2009-spring,54,0.537037,54.0,54.0,54.0,45.0,45.0,45.0
7,paradise,2009,summer,3.0,1.970000,1.165032,0.66,1.5100,2.360,2.6250,2.89,3.0,0.000000,0.000000,0.0,0.00,0.0,0.00,0.0,3.0,87.600000,2.433105,84.8,86.800,88.80,...,3.0,57.433333,4.271222,52.6,55.800,59.00,59.850,60.7,3.0,72.533333,3.365016,68.7,71.300,73.90,74.450,75.0,2009-summer,54,0.537037,54.0,54.0,45.0,45.0,45.0,45.0
8,paradise,2009,winter,3.0,1.106667,0.603435,0.70,0.7600,0.820,1.3100,1.80,3.0,5.400000,3.966106,1.0,3.75,6.5,7.60,8.7,3.0,55.800000,6.509224,49.1,52.650,56.20,...,3.0,25.433333,0.960902,24.4,25.000,25.60,25.950,26.3,3.0,40.600000,3.758989,36.7,38.800,40.90,42.550,44.2,2009-winter,54,0.537037,54.0,45.0,45.0,45.0,45.0,5.0
9,paradise,2010,fall,3.0,0.743333,0.610109,0.14,0.4350,0.730,1.0450,1.36,3.0,0.000000,0.000000,0.0,0.00,0.0,0.00,0.0,3.0,72.833333,10.327310,61.7,68.200,74.70,...,3.0,40.900000,12.517588,27.7,35.050,42.40,47.500,52.6,3.0,56.866667,11.437803,44.7,51.600,58.50,62.950,67.4,2010-fall,45,0.533333,54.0,45.0,45.0,45.0,5.0,5.0


## Correlations

In [25]:
def candidate(m,dv,placement=(1,1)):
    assert(dv in m.columns)
    return m[dv].sort_values().reset_index().iloc[placement[0]:placement[1]+1,:]

In [26]:
from functools import reduce

In [27]:
def topcorr(corrdf,lowestrank,dvs):
    candidates = [candidate(corrdf,dv,(1,lowestrank)) for dv in dvs]
    merger =  reduce(lambda x, y: pd.merge(x, y, on = 'index', how = 'outer'), candidates).fillna('--')
    return merger

In [28]:
#Dropping proportion of Females, but will put it back once I can order the y-axis
corrPortal_annual = df_reg_annual.loc[(df_reg_annual.source=='portal')]\
.drop(columns=['PRCP count', 'SNOW count', 'TMAX count', 'TMIN count', 'TAVG count','propFemale',
              'SNOW min', 'SNOW 25%', 'SNOW 50%',]).corr()
testx = corrPortal_annual.columns
testy = corrPortal_annual.index
testz = corrPortal_annual.values
test = go.Figure(go.Heatmap(x=testx,y=testy,z=testz))
# plot(test, filename = 'portal annual correlation matrix.html')
iplot(test, filename = 'portal annual correlation matrix.html')

In [29]:
mydvs = ['liznumberYear', 'popinYearplus1', 'popinYearless1', 'popinYearless2', 'popinYearless3', 'popinYearless4', 'popinYearless5']

In [30]:
annual =topcorr(corrPortal_annual,3,mydvs) 
annual

,index,liznumberYear,popinYearplus1,popinYearless1,popinYearless2,popinYearless3,popinYearless4,popinYearless5
0,TMIN 50%,-0.504974,-0.600148,--,--,--,--,--
1,TMIN mean,-0.49776,-0.62719,-0.457114,--,--,--,--
2,TMIN 25%,-0.446444,--,-0.467921,--,--,--,--
3,year,--,-0.616676,--,-0.289124,--,--,--
4,PRCP 75%,--,--,-0.429349,--,--,--,--
5,PRCP 50%,--,--,--,-0.665743,--,--,--
6,PRCP min,--,--,--,-0.463886,--,--,-0.411414
7,TMAX 50%,--,--,--,--,-0.330716,--,--
8,TMAX 25%,--,--,--,--,-0.323008,--,--
9,TMAX min,--,--,--,--,-0.276515,--,--


In [31]:
# print([key for key in seasons.keys() if seasons[key]=='fall'])
popvar = ['popinYearless1']#,'popinYearless2','popinYearless3','popinYearless4']
weathvars = annual['index'].tolist()
allvars = popvar+weathvars
threshold = .05

for var in allvars:
    tmp = df_reg_annual.loc[(df_reg_annual.source=='portal')][['liznumberYear',var]].dropna()
    try:
        r,p = stats.pearsonr(tmp['liznumberYear'],tmp[var])
        if p<=threshold:
            print('{} vs {}: r={}; p={}'.format('liznumberYear',var,r,p))
        else:
            print('{} vs {}: r={}; p= N.S.'.format('liznumberYear',var,r))
    except:
        popN = tmp['liznumberYear'].shape[0]
        popNA = tmp.shape[0]
        varN = tmp[var].shape[0]
        varNA = tmp.shape[0]
        print('Unable to calculate for values {}({} values; {} are NA) and {}({} values; {} are NA)'\
              .format('liznumberYear',popN,popNA,var, varN,varNA))

liznumberYear vs popinYearless1: r=0.4608603724974331; p= N.S.
liznumberYear vs TMIN 50%: r=-0.5049735888197354; p=0.03868891264394356
liznumberYear vs TMIN mean: r=-0.49776042252540714; p=0.04202827832034375
liznumberYear vs TMIN 25%: r=-0.44644408655380335; p= N.S.
liznumberYear vs year: r=-0.54745789748482; p=0.02292589302729298
liznumberYear vs PRCP 75%: r=-0.333891206396611; p= N.S.
liznumberYear vs PRCP 50%: r=-0.2810688167960773; p= N.S.
liznumberYear vs PRCP min: r=0.02972693136096969; p= N.S.
liznumberYear vs TMAX 50%: r=-0.42208069934771364; p= N.S.
liznumberYear vs TMAX 25%: r=-0.17733591496275997; p= N.S.
liznumberYear vs TMAX min: r=-0.08536573607324119; p= N.S.
liznumberYear vs TAVG std: r=-0.00946890748740143; p= N.S.
liznumberYear vs TMIN max: r=-0.4391703306007845; p= N.S.
liznumberYear vs TMAX std: r=0.10405372847051107; p= N.S.
liznumberYear vs SNOW mean: r=-0.0057260830081326125; p= N.S.
liznumberYear vs SNOW max: r=-0.10332163384830378; p= N.S.


In [32]:
# print([key for key in seasons.keys() if seasons[key]=='fall'])
popvar = ['liznumberYear']#,'popinYearless2','popinYearless3','popinYearless4']
weathvars = annual['index'].tolist()
allvars = popvar+weathvars
threshold = .05

for var in allvars:
    tmp = df_reg_annual.loc[(df_reg_annual.source=='portal')][['popinYearplus1',var]].dropna()
    try:
        r,p = stats.pearsonr(tmp['popinYearplus1'],tmp[var])
        if p<=threshold:
            print('{} vs {}: r={}; p={}'.format('popinYearplus1',var,r,p))
        else:
            None
    except:
        popN = tmp['popinYearplus1'].shape[0]
        popNA = tmp.shape[0]
        varN = tmp[var].shape[0]
        varNA = tmp.shape[0]
        print('Unable to calculate for values {}({} values; {} are NA) and {}({} values; {} are NA)'\
              .format('popinYearplus1',popN,popNA,var, varN,varNA))

popinYearplus1 vs TMIN 50%: r=-0.6001478743052835; p=0.013977346307322338
popinYearplus1 vs TMIN mean: r=-0.6271895387824434; p=0.009308429689284402
popinYearplus1 vs year: r=-0.6166759028550219; p=0.010948368276566577
popinYearplus1 vs TMAX 50%: r=-0.552260170912544; p=0.026538855567087422
popinYearplus1 vs TMIN max: r=-0.5106929049324576; p=0.04323393591626196


## To-Do
- Run MV correlation
    - IV should be pop at year 0
    - DV should be pop at year 1- year X plus abiotic factors
    - Which abiotic

## Season

### Spring

In [33]:
corrPortal_spring = df_reg_season.loc[(df_reg_season.source=='portal')&(df_reg_season.season.isin(['spring']))]\
.drop(columns=['PRCP count', 'SNOW count', 'TMAX count', 'TMIN count', 'TAVG count','propFemale',
              'SNOW min', 'SNOW 25%', 'SNOW 50%',]).corr()
testx = corrPortal_spring.columns
testy = corrPortal_spring.index
testz = corrPortal_spring.values
test = go.Figure(go.Heatmap(x=testx,y=testy,z=testz))
#plot(test, filename = 'portal spring correlation matrix.html')
iplot(test, filename = 'portal spring correlation matrix.html')

In [34]:
spring= topcorr(corrPortal_spring,3,mydvs) 
spring

,index,liznumberYear,popinYearplus1,popinYearless1,popinYearless2,popinYearless3,popinYearless4,popinYearless5
0,TMIN min,-0.418516,-0.418516,-0.418516,-0.376769,-0.159164,-0.159164,-0.159164
1,TMIN 25%,-0.389436,-0.389436,-0.389436,-0.343006,--,--,--
2,TAVG min,-0.382552,-0.382552,-0.382552,--,-0.0213942,-0.0213942,-0.0213942
3,PRCP std,--,--,--,-0.346456,-0.127367,-0.127367,-0.127367


In [35]:
print([key for key in seasons.keys() if seasons[key]=='spring'])
popvar = ['popinYearless1']#,'popinYearless2','popinYearless3','popinYearless4']
weathvars = spring['index'].tolist()
allvars = popvar+weathvars
threshold = .05

for var in allvars:
    tmp = df_reg_season.loc[(df_reg_season.source=='portal')&
                                               (df_reg_season.season.isin(['spring']))]\
                             [['liznumberYear',var]].dropna()
    try:
        r,p = stats.pearsonr(tmp['liznumberYear'],tmp[var])
        if p<=threshold:
            print('{} vs {}: r={}; p={}'.format('liznumberYear',var,r,p))
        else:
            print('{} vs {}: r={}; p= N.S.'.format('liznumberYear',var,r))
    except:
        popN = tmp['liznumberYear'].shape[0]
        popNA = tmp.shape[0]
        varN = tmp[var].shape[0]
        varNA = tmp.shape[0]
        print('Unable to calculate for values {}({} values; {} are NA) and {}({} values; {} are NA)'\
              .format('liznumberYear',popN,popNA,var, varN,varNA))

['Mar', 'Apr', 'May']
liznumberYear vs popinYearless1: r=0.9999999999999998; p=2.9707108702725584e-124
liznumberYear vs TMIN min: r=-0.41851647241248513; p= N.S.
liznumberYear vs TMIN 25%: r=-0.389435683851581; p= N.S.
liznumberYear vs TAVG min: r=-0.3825522632484052; p= N.S.
liznumberYear vs PRCP std: r=-0.22879382277879962; p= N.S.


In [36]:
print([key for key in seasons.keys() if seasons[key]=='spring'])
popvar = ['liznumberYear']#,'popinYearless2','popinYearless3','popinYearless4']
weathvars = spring['index'].tolist()
allvars = popvar+weathvars
threshold = .05

for var in allvars:
    tmp = df_reg_season.loc[(df_reg_season.source=='portal')&
                                               (df_reg_season.season.isin(['spring']))]\
                             [['popinYearplus1',var]].dropna()
    try:
        r,p = stats.pearsonr(tmp['popinYearplus1'],tmp[var])
        if p<=threshold:
            print('{} vs {}: r={}; p={}'.format('popinYearplus1',var,r,p))
        else:
            None
    except:
        popN = tmp['popinYearplus1'].shape[0]
        popNA = tmp.shape[0]
        varN = tmp[var].shape[0]
        varNA = tmp.shape[0]
        print('Unable to calculate for values {}({} values; {} are NA) and {}({} values; {} are NA)'\
              .format('popinYearplus1',popN,popNA,var, varN,varNA))

['Mar', 'Apr', 'May']
popinYearplus1 vs liznumberYear: r=0.9999999999999998; p=2.9707108702725584e-124


### Summer

In [37]:
corrPortal_summer = df_reg_season.loc[(df_reg_season.source=='portal')&(df_reg_season.season.isin(['summer']))]\
.drop(columns=['PRCP count', 'SNOW count', 'TMAX count', 'TMIN count', 'TAVG count','propFemale',
              'SNOW min', 'SNOW 25%', 'SNOW 50%',]).corr()
testx = corrPortal_summer.columns
testy = corrPortal_summer.index
testz = corrPortal_summer.values
test = go.Figure(go.Heatmap(x=testx,y=testy,z=testz))
#plot(test, filename = 'portal summer correlation matrix.html')
iplot(test, filename = 'portal summer correlation matrix.html')

In [38]:
summer= topcorr(corrPortal_summer,3,mydvs) 
summer

,index,liznumberYear,popinYearplus1,popinYearless1,popinYearless2,popinYearless3,popinYearless4,popinYearless5
0,year,-0.571651,-0.571651,-0.547458,-0.455601,-0.455601,-0.455601,-0.416456
1,TMIN 75%,-0.546627,-0.546627,-0.539115,--,--,--,--
2,TMIN 50%,-0.529302,-0.529302,-0.556413,--,--,--,--
3,PRCP 50%,--,--,--,-0.451575,-0.451575,-0.451575,-0.473222
4,TAVG max,--,--,--,-0.434557,-0.434557,-0.434557,-0.392816


In [39]:
print([key for key in seasons.keys() if seasons[key]=='summer'])
popvar = ['liznumberYear']#,'popinYearless2','popinYearless3','popinYearless4']
weathvars = summer['index'].tolist()
allvars = popvar+weathvars
threshold = .05

for var in allvars:
    tmp = df_reg_season.loc[(df_reg_season.source=='portal')&
                                               (df_reg_season.season.isin(['summer']))]\
                             [['popinYearplus1',var]].dropna()
    try:
        r,p = stats.pearsonr(tmp['popinYearplus1'],tmp[var])
        if p<=threshold:
            print('{} vs {}: r={}; p={}'.format('popinYearplus1',var,r,p))
        else:
            None
    except:
        popN = tmp['popinYearplus1'].shape[0]
        popNA = tmp.shape[0]
        varN = tmp[var].shape[0]
        varNA = tmp.shape[0]
        print('Unable to calculate for values {}({} values; {} are NA) and {}({} values; {} are NA)'\
              .format('popinYearplus1',popN,popNA,var, varN,varNA))

['Jun', 'Jul', 'Aug']
popinYearplus1 vs liznumberYear: r=0.9999999999999998; p=2.9707108702725584e-124
popinYearplus1 vs year: r=-0.5716511155042829; p=0.013194217739335838
popinYearplus1 vs TMIN 75%: r=-0.546627242213519; p=0.018907757879229714
popinYearplus1 vs TMIN 50%: r=-0.5293021584412356; p=0.023892702568078714


In [56]:
print([key for key in seasons.keys() if seasons[key]=='summer'])
popvar = ['popinYearless1']#,'popinYearless2','popinYearless3','popinYearless4']
weathvars = summer['index'].tolist()
allvars = popvar+weathvars
threshold = .05

for var in allvars:
    tmp = df_reg_season.loc[(df_reg_season.source=='portal')&
                                               (df_reg_season.season.isin(['summer']))]\
                             [['liznumberYear',var]].dropna()
    try:
        r,p = stats.pearsonr(tmp['liznumberYear'],tmp[var])
        if p<=threshold:
            print('{} vs {}: r={}; p={}'.format('liznumberYear',var,r,p))
        else:
            None
    except:
        popN = tmp['liznumberYear'].shape[0]
        popNA = tmp.shape[0]
        varN = tmp[var].shape[0]
        varNA = tmp.shape[0]
        print('Unable to calculate for values {}({} values; {} are NA) and {}({} values; {} are NA)'\
              .format('liznumberYear',popN,popNA,var, varN,varNA))

['Jun', 'Jul', 'Aug']
liznumberYear vs popinYearless1: r=1.0; p=0.0
liznumberYear vs year: r=-0.5716511155042829; p=0.013194217739335838
liznumberYear vs TMIN 75%: r=-0.546627242213519; p=0.018907757879229714
liznumberYear vs TMIN 50%: r=-0.5293021584412356; p=0.023892702568078714


### Fall

In [41]:
corrPortal_fall = df_reg_season.loc[(df_reg_season.source=='portal')&(df_reg_season.season.isin(['fall']))]\
.drop(columns=['PRCP count', 'SNOW count', 'TMAX count', 'TMIN count', 'TAVG count','propFemale',
              'SNOW min', 'SNOW 25%', 'SNOW 50%',]).corr()
testx = corrPortal_fall.columns
testy = corrPortal_fall.index
testz = corrPortal_fall.values
test = go.Figure(go.Heatmap(x=testx,y=testy,z=testz))
# plot(test, filename = 'portal fall correlation matrix.html')
iplot(test, filename = 'portal fall correlation matrix.html')

In [42]:
fall= topcorr(corrPortal_fall,3,mydvs) 
fall

,index,liznumberYear,popinYearplus1,popinYearless1,popinYearless2,popinYearless3,popinYearless4,popinYearless5
0,TAVG 50%,-0.483759,--,-0.483759,-0.483759,-0.454457,--,--
1,TMAX 25%,-0.473299,--,-0.473299,-0.473299,-0.43952,--,--
2,TAVG 25%,-0.463329,--,-0.463329,-0.463329,-0.432123,--,--
3,TMIN mean,--,-0.536774,--,--,--,--,--
4,TMIN 50%,--,-0.496319,--,--,--,--,--
5,TAVG mean,--,-0.490713,--,--,--,--,--
6,TMAX mean,--,--,--,--,--,-0.331657,-0.331657
7,TMAX 75%,--,--,--,--,--,-0.318222,-0.318222
8,TAVG 75%,--,--,--,--,--,-0.31738,-0.31738


In [43]:
print([key for key in seasons.keys() if seasons[key]=='fall'])
popvar = ['liznumberYear']#,'popinYearless2','popinYearless3','popinYearless4']
weathvars = fall['index'].tolist()
allvars = popvar+weathvars
threshold = .05

for var in allvars:
    tmp = df_reg_season.loc[(df_reg_season.source=='portal')&
                                               (df_reg_season.season.isin(['fall']))]\
                             [['popinYearplus1',var]].dropna()
    try:
        r,p = stats.pearsonr(tmp['popinYearplus1'],tmp[var])
        if p<=threshold:
            print('{} vs {}: r={}; p={}'.format('popinYearplus1',var,r,p))
        else:
            None
    except:
        popN = tmp['popinYearplus1'].shape[0]
        popNA = tmp.shape[0]
        varN = tmp[var].shape[0]
        varNA = tmp.shape[0]
        print('Unable to calculate for values {}({} values; {} are NA) and {}({} values; {} are NA)'\
              .format('popinYearplus1',popN,popNA,var, varN,varNA))

['Sept', 'Oct', 'Nov']
popinYearplus1 vs TMIN mean: r=-0.5367742004258258; p=0.026308733367876744
popinYearplus1 vs TMIN 50%: r=-0.49631911123997824; p=0.04272090253383262
popinYearplus1 vs TAVG mean: r=-0.49071252815725463; p=0.04549753258648353


In [44]:
print([key for key in seasons.keys() if seasons[key]=='fall'])
popvar = ['popinYearless1']#,'popinYearless2','popinYearless3','popinYearless4']
weathvars = fall['index'].tolist()
allvars = popvar+weathvars
threshold = .05

for var in allvars:
    tmp = df_reg_season.loc[(df_reg_season.source=='portal')&
                                               (df_reg_season.season.isin(['fall']))]\
                             [['liznumberYear',var]].dropna()
    try:
        r,p = stats.pearsonr(tmp['liznumberYear'],tmp[var])
        if p<=threshold:
            print('{} vs {}: r={}; p={}'.format('liznumberYear',var,r,p))
        else:
            None
    except:
        popN = tmp['liznumberYear'].shape[0]
        popNA = tmp.shape[0]
        varN = tmp[var].shape[0]
        varNA = tmp.shape[0]
        print('Unable to calculate for values {}({} values; {} are NA) and {}({} values; {} are NA)'\
              .format('liznumberYear',popN,popNA,var, varN,varNA))

['Sept', 'Oct', 'Nov']
liznumberYear vs popinYearless1: r=0.9999999999999998; p=2.9707108702725584e-124
liznumberYear vs TAVG 50%: r=-0.4837585273722812; p=0.04194477396262829
liznumberYear vs TMAX 25%: r=-0.4732987467611391; p=0.04726897888016859


### Winter


In [45]:
corrPortal_winter = df_reg_season.loc[(df_reg_season.source=='portal')&(df_reg_season.season.isin(['winter']))]\
.drop(columns=['PRCP count', 'SNOW count', 'TMAX count', 'TMIN count', 'TAVG count','propFemale',
              'SNOW min', 'SNOW 25%', 'SNOW 50%',]).corr()
testx = corrPortal_winter.columns
testy = corrPortal_winter.index
testz = corrPortal_winter.values
test = go.Figure(go.Heatmap(x=testx,y=testy,z=testz))
#plot(test, filename = 'portal winter correlation matrix.html')
iplot(test, filename = 'portal winter correlation matrix.html')

In [46]:
winter= topcorr(corrPortal_winter,3,mydvs) 
winter

,index,liznumberYear,popinYearplus1,popinYearless1,popinYearless2,popinYearless3,popinYearless4,popinYearless5
0,TMIN std,-0.321208,-0.321208,--,--,--,--,--
1,TAVG min,-0.253074,-0.253074,--,--,--,--,--
2,TAVG 25%,-0.22446,-0.22446,--,--,--,--,--
3,PRCP 25%,--,--,-0.550567,-0.550567,-0.550567,--,--
4,TMIN 25%,--,--,-0.516355,-0.516355,-0.516355,-0.50074,--
5,TMIN mean,--,--,-0.509694,-0.509694,-0.509694,-0.494335,--
6,PRCP 50%,--,--,--,--,--,-0.55685,--
7,PRCP 75%,--,--,--,--,--,--,-0.259542
8,PRCP mean,--,--,--,--,--,--,-0.259291
9,PRCP max,--,--,--,--,--,--,-0.246679


In [47]:
print([key for key in seasons.keys() if seasons[key]=='winter'])
popvar = ['liznumberYear']#,'popinYearless2','popinYearless3','popinYearless4']
weathvars = winter['index'].tolist()
allvars = popvar+weathvars
threshold = .05

for var in allvars:
    tmp = df_reg_season.loc[(df_reg_season.source=='portal')&
                                               (df_reg_season.season.isin(['winter']))]\
                             [['popinYearplus1',var]].dropna()
    try:
        r,p = stats.pearsonr(tmp['popinYearplus1'],tmp[var])
        if p<=threshold:
            print('{} vs {}: r={}; p={}'.format('popinYearplus1',var,r,p))
        else:
            None
    except:
        popN = tmp['popinYearplus1'].shape[0]
        popNA = tmp.shape[0]
        varN = tmp[var].shape[0]
        varNA = tmp.shape[0]
        print('Unable to calculate for values {}({} values; {} are NA) and {}({} values; {} are NA)'\
              .format('popinYearplus1',popN,popNA,var, varN,varNA))

['Dec', 'Jan', 'Feb']
popinYearplus1 vs liznumberYear: r=1.0; p=0.0


Drop

In [55]:
print([key for key in seasons.keys() if seasons[key]=='winter'])
popvar = ['popinYearless1']#,'popinYearless2','popinYearless3','popinYearless4']
weathvars = winter['index'].tolist()
allvars = popvar+weathvars
threshold = .05

for var in allvars:
    tmp = df_reg_season.loc[(df_reg_season.source=='portal')&
                                               (df_reg_season.season.isin(['winter']))]\
                             [['liznumberYear',var]].dropna()
    try:
        r,p = stats.pearsonr(tmp['liznumberYear'],tmp[var])
        if p<=threshold:
            print('{} vs {}: r={}; p={}'.format('liznumberYear',var,r,p))
        else:
            None
    except:
        popN = tmp['liznumberYear'].shape[0]
        popNA = tmp.shape[0]
        varN = tmp[var].shape[0]
        varNA = tmp.shape[0]
        print('Unable to calculate for values {}({} values; {} are NA) and {}({} values; {} are NA)'\
              .format('liznumberYear',popN,popNA,var, varN,varNA))

['Dec', 'Jan', 'Feb']


Unlike the other season, winter precipitation in year 0 has correlalations for population size in subsequent years.

# Resume Here

[Back to TOC](#Table-of-Contents)

Need to model this with regression.
two predictors: pop in year weather in year
dv: pop in year 2

Train model using captured juveniles to predict age class 
include weather

In [49]:
import numpy as np
import pingouin as pg
from scipy import stats

In [50]:
mydvs = ['liznumberYear', 'popinYearless1']#, 'popinYearless2', 'popinYearless3', 'popinYearless4', 'popinYearless5']

In [51]:
popVar = 'liznumberYear'
summerPred = df_reg_season.loc[(df_reg_season.source=='portal')&
                                               (df_reg_season.season.isin(['summer']))]\
                             [['popinYearplus1','TMIN 50%']].dropna()

sjWsumm = pg.linear_regression(summerPred[['TMIN 50%']],
                     summerPred['popinYearplus1'],
                     remove_na=True)

sjWsumm

,names,coef,se,T,pval,r2,adj_r2,CI[2.5%],CI[97.5%]
0,Intercept,657.031303,241.172590,2.724320,0.015005,0.280161,0.235171,145.768251,1168.294354
1,TMIN 50%,-10.759626,4.311727,-2.495433,0.023893,0.280161,0.235171,-19.900079,-1.619173


In [52]:
popVar = 'liznumberYear'
summerPred = df_reg_season.loc[(df_reg_season.source=='portal')&
                                               (df_reg_season.season.isin(['summer']))]\
                             [['popinYearplus1','TMIN 50%']].dropna()

sjWsumm = pg.linear_regression(summerPred[['TMIN 50%']],
                     summerPred['popinYearplus1'],
                     remove_na=True)

sjWsumm

,names,coef,se,T,pval,r2,adj_r2,CI[2.5%],CI[97.5%]
0,Intercept,657.031303,241.172590,2.724320,0.015005,0.280161,0.235171,145.768251,1168.294354
1,TMIN 50%,-10.759626,4.311727,-2.495433,0.023893,0.280161,0.235171,-19.900079,-1.619173


In [53]:
popVar = 'liznumberYear'
fallPred = df_reg_season.loc[(df_reg_season.source=='portal')&
                                               (df_reg_season.season.isin(['fall']))]\
                             [['popinYearplus1','TMIN 50%']].dropna()

sjWfall = pg.linear_regression(fallPred[['TMIN 50%']],
                     fallPred['popinYearplus1'],
                     remove_na=True)

sjWfall

,names,coef,se,T,pval,r2,adj_r2,CI[2.5%],CI[97.5%]
0,Intercept,235.229775,80.855205,2.909272,0.010790,0.246333,0.196088,62.890985,407.568565
1,TMIN 50%,-4.275817,1.931089,-2.214200,0.042721,0.246333,0.196088,-8.391837,-0.159798


## Growth

## Sex Ratio